# House prices: Advanced Regression Techniques

This is my code for (this Kaggle competition)[https://www.kaggle.com/c/house-prices-advanced-regression-techniques]

## Setup

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [131]:
from fastai.imports import *
from fastai.structured import *
from fastai.column_data import *

In [44]:
# With this line all the columns will be displayed
pd.options.display.max_columns = None

In [4]:
PATH = os.path.join('data')

## Loading data

In [13]:
train_df = pd.read_csv(os.path.join(PATH, "train.csv"))
test_df = pd.read_csv(os.path.join(PATH, "test.csv"))

In [18]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [43]:
DataFrameSummary(train_df).summary()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460,1460,NaN,1201,1460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1460,1460,1460,1460,NaN,NaN,NaN,NaN,NaN,1452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1460,NaN,1460,1460,1460,NaN,NaN,NaN,NaN,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,NaN,1460,NaN,1460,NaN,NaN,1379,NaN,1460,1460,NaN,NaN,NaN,1460,1460,1460,1460,1460,1460,NaN,NaN,NaN,1460,1460,1460,NaN,NaN,1460
mean,730.5,56.8973,NaN,70.05,10516.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.09932,5.57534,1971.27,1984.87,NaN,NaN,NaN,NaN,NaN,103.685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443.64,NaN,46.5493,567.24,1057.43,NaN,NaN,NaN,NaN,1162.63,346.992,5.84452,1515.46,0.425342,0.0575342,1.56507,0.382877,2.86644,1.04658,NaN,6.51781,NaN,0.613014,NaN,NaN,1978.51,NaN,1.76712,472.98,NaN,NaN,NaN,94.2445,46.6603,21.9541,3.40959,15.061,2.7589,NaN,NaN,NaN,43.489,6.32192,2007.82,NaN,NaN,180921
std,421.61,42.3006,NaN,24.2848,9981.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.383,1.1128,30.2029,20.6454,NaN,NaN,NaN,NaN,NaN,181.066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,456.098,NaN,161.319,441.867,438.705,NaN,NaN,NaN,NaN,386.588,436.528,48.6231,525.48,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,NaN,1.62539,NaN,0.644666,NaN,NaN,24.6897,NaN,0.747315,213.805,NaN,NaN,NaN,125.339,66.256,61.1191,29.3173,55.7574,40.1773,NaN,NaN,NaN,496.123,2.70363,1.3281,NaN,NaN,79442.5
min,1,20,NaN,21,1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1872,1950,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,NaN,NaN,NaN,NaN,334,0,0,334,0,0,0,0,0,0,NaN,2,NaN,0,NaN,NaN,1900,NaN,0,0,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,0,1,2006,NaN,NaN,34900
25%,365.75,20,NaN,59,7553.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,1954,1967,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,223,795.75,NaN,NaN,NaN,NaN,882,0,0,1129.5,0,0,1,0,2,1,NaN,5,NaN,0,NaN,NaN,1961,NaN,1,334.5,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,NaN,NaN,129975
50%,730.5,50,NaN,69,9478.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,5,1973,1994,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,383.5,NaN,0,477.5,991.5,NaN,NaN,NaN,NaN,1087,0,0,1464,0,0,2,0,3,1,NaN,6,NaN,1,NaN,NaN,1980,NaN,2,480,NaN,NaN,NaN,0,25,0,0,0,0,NaN,NaN,NaN,0,6,2008,NaN,NaN,163000
75%,1095.25,70,NaN,80,11601.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,6,2000,2004,NaN,NaN,NaN,NaN,NaN,166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,712.25,NaN,0,808,1298.25,NaN,NaN,NaN,NaN,1391.25,728,0,1776.75,1,0,2,1,3,1,NaN,7,NaN,1,NaN,NaN,2002,NaN,2,576,NaN,NaN,NaN,168,68,0,0,0,0,NaN,NaN,NaN,0,8,2009,NaN,NaN,214000
max,1460,190,NaN,313,215245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,9,2010,2010,NaN,NaN,NaN,NaN,NaN,1600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5644,NaN,1474,2336,6110,NaN,NaN,NaN,NaN,4692,2065,572,5642,3,2,3,2,8,3,NaN,14,NaN,3,NaN,NaN,2010,NaN,4,1418,NaN,NaN,NaN,857,547,552,508,480,738,NaN,NaN,NaN,15500,12,2010,NaN,NaN,755000
counts,1460,1460,1460,1201,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1452,1452,1460,1460,1460,1423,1423,1422,1423,1460,1422,1460,1460,1460,1460,1460,1460,1459,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,770,137

In [16]:
(len(training_df), len(test_df))

(1460, 1459)

## Feature engineering

I remove Id since it will not go into the model as input.

In [116]:
del train_df["Id"]

I went through all the variables and classified them either as categorical or continuous.
I decided not to include some of them for the following reasons:
- "Alley": it has 93.77% of missing values.
- "Utilities": there only one case of value different than "AllPub". It's to little data for the model to generalize.
- "MiscFeature": it has 96.30% of missing values and "MiscVal" should give the sufficient information to the model


In [120]:
del train_df["Alley"]
del train_df["Utilities"]
del train_df["MiscFeature"]

In [106]:
categorical_variables = ["MSSubClass", "MSZoning", "Street", "LotShape", "LandContour",
                         "LotConfig", "LandSlope", "Neighborhood", "Condition1", "Condition2",
                        "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", 
                         "Exterior2nd", "MasVnrType", "ExterQual", "ExterCond", "Foundation", 
                         "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
                        "Heating",  "HeatingQC", "CentralAir", "Electrical", "KitchenQual", 
                         "Functional", "Fireplaces", "FireplaceQu", "GarageType", "GarageFinish",
                        "GarageCars", "GarageQual", "GarageCond", "PavedDrive", "PoolQC",
                        "Fence", "MoSold", "YrSold", "SaleType", "SaleCondition"]
continuous_variables = ["LotFrontage", "LotArea", "OverallQual", "OverallCond", 
                        "YearBuilt", "YearRemodAdd", "MasVnrArea", "BsmtFinSF1", "BsmtUnfSF",
                       "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
                        "BsmtFullBath", "BsmtHalfBath", "FullBath","HalfBath","BedroomAbvGr",
                       "KitchenAbvGr", "TotRmsAbvGrd", "GarageYrBlt", "GarageArea",
                       "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
                        "ScreenPorch", "PoolArea", "MiscVal"]

have_missing = ["LotFrontage", "BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1", 
                "BsmtFinType2", "FireplaceQu", "GarageYrBlt", "GarageQual", "GarageCond",
               "PoolQC"]

In [122]:
# Filling in missing values
train_df["LotFrontage"] = train_df["LotFrontage"].fillna(0)
train_df["BsmtQual"] = train_df["BsmtQual"].fillna("NA")
train_df["BsmtCond"] = train_df["BsmtCond"].fillna("NA")
train_df["BsmtExposure"] = train_df["BsmtExposure"].fillna("NA")
train_df["BsmtFinType1"] = train_df["BsmtFinType1"].fillna("NA")
train_df["BsmtFinType2"] = train_df["BsmtFinType2"].fillna("NA")
train_df["FireplaceQu"] = train_df["FireplaceQu"].fillna("NA")
train_df["GarageYrBlt"] = train_df["GarageYrBlt"].fillna(1800)
train_df["GarageQual"] = train_df["GarageQual"].fillna("NA")
train_df["GarageCond"] = train_df["GarageCond"].fillna("NA")
train_df["PoolQC"] = train_df["PoolQC"].fillna("NA")

In [121]:
for variable in categorical_variables: train_df[variable] = train_df[variable].astype('category').cat.as_ordered()

In [124]:
for variable in continuous_variables:
    train_df[variable] = train_df[variable].astype('float32')


In [129]:
n = len(train_df)
idxs = get_cv_idxs(n, val_pct=0.3)
train_sample = train_df.iloc[idxs]
sample_size = len(train_sample); sample_size


438

In [133]:
dl_train_df, y, nas, mapper = proc_df(train_sample, 'SalePrice', do_scale=True)
yl = np.log(y)

In [134]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(sample_size * train_ratio); train_size
val_idx = list(range(train_size, len(dl_train_df)))


## Learning model

In [135]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)


In [145]:
model = ColumnarModelData.from_data_frame(PATH, val_idx, dl_train_df, yl.astype(np.float32), cat_flds=categorical_variables, bs=128)

In [146]:
categories_sizes = [(c, len(train_sample[c].cat.categories)) for c in categorical_variables]

In [148]:
embedding_sizes = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [149]:
m = md.get_learner(embedding_sizes, len(dl_train_df.columns)-len(categorical_variables),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [151]:
m.lr_find()
m.sched.plot(100)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))


  0%|          | 0/3 [00:00<?, ?it/s]


RuntimeError: index out of range at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/TH/generic/THTensorMath.c:277